# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных** 

Таблица `books`

Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц; 
- `publication_date` — дата публикации книги; 
- `publisher_id` — идентификатор издателя.

Таблица `authors`

Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора

Таблица `publishers`

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства; 

Таблица `ratings`

Содержит данные о пользовательских оценках книг: 
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку; 
- `rating` — оценка книги.

Таблица `reviews`

Содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор; 
- `text` — текст обзора.

## Исследование таблиц — вывод первых строк

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': '***', # пароль
             'host': '***',
             'port': 1111, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
query = '''
            SELECT * FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''
            SELECT * FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''
            SELECT * FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''
            SELECT * FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''
            SELECT * FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [8]:
query_1 = '''
            SELECT COUNT(book_id) 
            FROM books
            WHERE publication_date > '2000-01-01'
        '''
pd.io.sql.read_sql(query_1, con = engine)

,count
0,819


**Вывод**

819 книг вышло после 1 января 2000 года

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [9]:
query_2 = '''
            WITH 
            re AS ( SELECT  b.title,
                            b.book_id,
                    COUNT(rev.review_id) AS count_reviews
                    FROM books as b
                    FULL OUTER JOIN reviews AS rev ON b.book_id = rev.book_id
                    GROUP BY b.title, b.book_id)
            
            SELECT  re.title,
                    re.count_reviews,
                    ROUND(AVG(r.rating), 2) AS avg_rating
            FROM re
            FULL OUTER JOIN ratings AS r ON re.book_id = r.book_id
            GROUP BY re.title, re.count_reviews
            
        '''
pd.io.sql.read_sql(query_2, con = engine)

,title,count_reviews,avg_rating
0,V for Vendetta,3,4.22
1,The Fiery Cross (Outlander #5),4,4.20
2,The Canterbury Tales,3,3.33
3,Burning Chrome (Sprawl #0),2,4.50
4,The Summons,3,4.33
...,...,...,...
995,The Notebook (The Notebook #1),5,3.87
996,The Innocent Man: Murder and Injustice in a Sm...,2,4.50
997,Rules of Prey (Lucas Davenport #1),3,3.33
998,A Hunger Like No Other (Immortals After Dark #1),2,4.00


**Вывод**

Мы получили таблицу 1000 строк - количесвто книг в нашей базе, посчитали количество обзоров `count_reviews` для каждой книги, и среднюю пользовательскую оценку `avg_rating`

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
query_3 = '''
            SELECT  p.publisher,
                    COUNT(b.book_id)
            FROM publishers as p
            LEFT JOIN books AS b ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY COUNT(b.book_id) DESC
        '''
pd.io.sql.read_sql(query_3, con = engine)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


**Вывод**

Издательство, которое выпустило наибольшее число книг толще 50 страниц - `Penguin Books	`

Чтобы удостоверится что сортировка работает правильно вывела все издательства где есть книги больше 50 страниц, и само количество книг

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [11]:
query_4 = '''
            
            WITH
            br AS (SELECT  a.author,
                           ROUND(AVG(r.rating), 2) as avg_book_rating
                   FROM authors as a
                   LEFT JOIN books AS b ON b.author_id = a.author_id
                   LEFT JOIN ratings AS r ON r.book_id = b.book_id
                   GROUP BY b.book_id, a.author
                   having COUNT(r.rating) >= 50)
            
            SELECT br.author,
                   AVG(avg_book_rating)
            FROM br
            GROUP BY br.author
            ORDER BY AVG(avg_book_rating) DESC
        '''
pd.io.sql.read_sql(query_4, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.285
1,Markus Zusak/Cao Xuân Việt Khương,4.260
2,J.R.R. Tolkien,4.260
3,Louisa May Alcott,4.190
4,Rick Riordan,4.080
5,William Golding,3.900
6,J.D. Salinger,3.830
7,William Shakespeare/Paul Werstine/Barbara A. M...,3.790
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.790
9,Dan Brown,3.755


**Вывод**

Автор с самой высокой средней оценкой книг - `J.K. Rowling/Mary GrandPré`

Чтобы удостоверится что сортировка работает правильно вывела всех авторов книг с более 50 оценок, и саму среднюю оценку

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query_5 = '''
            
            WITH
            u_r AS (SELECT  username  
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) >= 50),
            
            rev AS (SELECT username,
                           COUNT(review_id) as rev_count 
                    FROM reviews
                    GROUP BY username)
           
           SELECT ROUND(AVG(rev.rev_count))
            
           FROM u_r
           JOIN rev ON u_r.username = rev.username
           WHERE rev.username IN (u_r.username)
           
        '''
pd.io.sql.read_sql(query_5, con = engine)

,round
0,24.0


**Вывод**

Пользователей, поставивших больше 50 оценок всего 9. Их среднее число рецензий 24.

In [13]:
query = '''
        WITH
        i as(SELECT b.title as title, b.book_id as book_id, AVG(ra.rating) as rating
            FROM books as b
            LEFT JOIN ratings as ra ON b.book_id = ra.book_id
            GROUP BY b.book_id )
            
        SELECT i.title, i.rating, COUNT(re.review_id)
        FROM i LEFT JOIN reviews as re ON i.book_id = re.book_id
        GROUP BY i.title, i.rating
        '''
pd.io.sql. read_sql (query, con = engine)


,title,rating,count
0,Far From the Madding Crowd,4.250000,3
1,2nd Chance (Women's Murder Club #2),3.000000,3
2,Fullmetal Alchemist Vol. 1 (Fullmetal Alchemi...,4.250000,2
3,Mystic River,4.250000,3
4,Suzanne's Diary for Nicholas,4.600000,3
...,...,...,...
995,Year of Wonders,3.200000,4
996,The House of Mirth,4.000000,3
997,True Believer (Jeremy Marsh & Lexie Darnell #1),4.333333,3
998,Goodnight Nobody,3.500000,2
